<a href="https://colab.research.google.com/github/sheunq/Telco-customer-churn-prediction/blob/main/Telco_customer_churn_prediction_with_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=a9fa8b788c0bf617d981c41aedcc3936952f3a2eb06eaf35a523b9b85af0cd57
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [2]:
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorIndexer

In [3]:
spark=SparkSession.builder.getOrCreate()
df=spark.read.csv('/content/drive/MyDrive/Colab Notebooks/WA_Fn-UseC_-Telco-Customer-Churn.csv',header=True,inferSchema=True)
df.show()

+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|7590-VHVEG|Female|            0|    Yes|        No|     1|  

In [4]:
df=df.drop('customerID')

In [5]:
df.filter(df.gender.isNull()).show()

+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+---

In [6]:
df.printSchema()

root
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [7]:
categorical=[[df.dtypes[i] for i in range(len(df.dtypes)) if df.dtypes[i][1]=='string'][i][0] for i in range(len([df.dtypes[i] for i in range(len(df.dtypes)) if df.dtypes[i][1]=='string']))]
categorical

['gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges',
 'Churn']

In [8]:
sid=StringIndexer(inputCols=categorical,outputCols=[categorical[i]+'_indexer' for i in range(len(categorical))])
sid=sid.fit(df)
sid=sid.transform(df)
sid.show(5)

+------+-------------+-------+----------+------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+--------------------+--------------+------------+-----+--------------+---------------+------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------+------------------------+---------------------+--------------------+-------------+
|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|gender_indexer|Partner_indexer|Dependents_indexer|PhoneService_indexer|MultipleLines_indexer|InternetService_indexer|O

In [9]:
sid_new=sid.drop('gender',
 'Partner',
 'Dependents',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'TotalCharges',
 'Churn')
sid_new.show(3)

+-------------+------+--------------+--------------+---------------+------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------+------------------------+---------------------+--------------------+-------------+
|SeniorCitizen|tenure|MonthlyCharges|gender_indexer|Partner_indexer|Dependents_indexer|PhoneService_indexer|MultipleLines_indexer|InternetService_indexer|OnlineSecurity_indexer|OnlineBackup_indexer|DeviceProtection_indexer|TechSupport_indexer|StreamingTV_indexer|StreamingMovies_indexer|Contract_indexer|PaperlessBilling_indexer|PaymentMethod_indexer|TotalCharges_indexer|Churn_indexer|
+-------------+------+--------------+--------------+---------------+------------------+--------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+---

In [10]:
train,test=sid_new.randomSplit([0.7,0.3],seed=123)

In [38]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier#,DecisionTreeClassifier

In [39]:
vec=VectorAssembler(inputCols=sid_new.drop('Churn_indexer').columns,outputCol='features')
#paramGrid = ParamGridBuilder().addGrid(dt.maxBins, [49, 52, 55]).addGrid(dt.maxDepth, [4, 6, 8]).addGrid(rf.impurity, ["entropy", "gini"]).build()
model=LogisticRegression(featuresCol='features',labelCol='Churn_indexer')#(impurity='gini', maxDepth=5, maxBins=1000000000,featuresCol = 'features', labelCol = 'Credit_Score_indexer')
pipeline = Pipeline(stages = [vec, model])

In [40]:
pipe=pipeline.fit(train)
pred=pipe.transform(test)

In [41]:
pred.printSchema()

root
 |-- SeniorCitizen: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- gender_indexer: double (nullable = false)
 |-- Partner_indexer: double (nullable = false)
 |-- Dependents_indexer: double (nullable = false)
 |-- PhoneService_indexer: double (nullable = false)
 |-- MultipleLines_indexer: double (nullable = false)
 |-- InternetService_indexer: double (nullable = false)
 |-- OnlineSecurity_indexer: double (nullable = false)
 |-- OnlineBackup_indexer: double (nullable = false)
 |-- DeviceProtection_indexer: double (nullable = false)
 |-- TechSupport_indexer: double (nullable = false)
 |-- StreamingTV_indexer: double (nullable = false)
 |-- StreamingMovies_indexer: double (nullable = false)
 |-- Contract_indexer: double (nullable = false)
 |-- PaperlessBilling_indexer: double (nullable = false)
 |-- PaymentMethod_indexer: double (nullable = false)
 |-- TotalCharges_indexer: double (nullable = false)
 |-- Churn_indexe

In [42]:
pred.select('features','rawPrediction','probability','prediction').show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[0.0,0.0,20.0,1.0...|[2.18151149971421...|[0.89857690783103...|       0.0|
|[0.0,0.0,52.55,1....|[1.08740331920282...|[0.74789243438193...|       0.0|
|[0.0,0.0,73.35,1....|[1.09111151016285...|[0.74859096842803...|       0.0|
|(19,[1,2,8,9,10,1...|[1.30736601498613...|[0.78707206108495...|       0.0|
|[0.0,1.0,18.85,1....|[1.44975652595262...|[0.80996096026419...|       0.0|
|[0.0,1.0,19.1,1.0...|[1.23468836615246...|[0.77463809442968...|       0.0|
|[0.0,1.0,19.2,0.0...|[1.54728942159643...|[0.82452189525312...|       0.0|
|[0.0,1.0,19.3,0.0...|[1.92631103348621...|[0.87284054457817...|       0.0|
|[0.0,1.0,19.3,0.0...|[1.54829283812300...|[0.82466702784281...|       0.0|
|[0.0,1.0,19.4,1.0...|[1.23745969646354...|[0.77512152816058...|       0.0|
|[0.0,1.0,19

In [16]:
pred.take(1)

[Row(SeniorCitizen=0, tenure=0, MonthlyCharges=20.0, gender_indexer=1.0, Partner_indexer=1.0, Dependents_indexer=1.0, PhoneService_indexer=0.0, MultipleLines_indexer=0.0, InternetService_indexer=2.0, OnlineSecurity_indexer=2.0, OnlineBackup_indexer=2.0, DeviceProtection_indexer=2.0, TechSupport_indexer=2.0, StreamingTV_indexer=2.0, StreamingMovies_indexer=2.0, Contract_indexer=1.0, PaperlessBilling_indexer=1.0, PaymentMethod_indexer=1.0, TotalCharges_indexer=0.0, Churn_indexer=0.0, features=DenseVector([0.0, 0.0, 20.0, 1.0, 1.0, 1.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 0.0]), rawPrediction=DenseVector([2.1815, -2.1815]), probability=DenseVector([0.8986, 0.1014]), prediction=0.0)]

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol = 'Churn_indexer', rawPredictionCol = 'rawPrediction')
print( f'{evaluator.getMetricName()}: {evaluator.evaluate(pred)}')

areaUnderROC: 0.8438195747342131


In [18]:
print(model.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: Churn_indexer)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimizat

In [43]:
vect=VectorAssembler(inputCols=sid_new.drop('Churn_indexer').columns,outputCol='features')
vect_train=vect.transform(train)
vect_test=vect.transform(test)

In [44]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(model.regParam, [0.01, 0.5, 2.0])#0.01, 0.5, 2.0
             .addGrid(model.elasticNetParam,[0.0, 0.5, 1.0])#[0.0, 0.5, 1.0]
             .addGrid(model.maxIter, [1, 5, 10])
             .build())

In [45]:
cv = CrossValidator(estimator=model, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(vect_train.select('features','Churn_indexer'))

In [46]:
#cvModel.transform(pred.select('features','Churn_indexer'))
evaluator.evaluate(cvModel.transform(vect_test.select('features','Churn_indexer')))

0.842537002293099